In [40]:
import os

In [41]:
%pwd

'd:\\chicken_fecal_clasification'

In [42]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [43]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [44]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

    

In [45]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

In [51]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downlaod! with the folowwing info: \n{headers}")
        else:
            logger.info(f"Data file already exists at :{get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [52]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-23 12:11:33,506] {d:\chicken_fecal_clasification\src\cnnClassifier\utils\common.py:24} INFO - yaml file: config\config.yaml read successfully
[2024-05-23 12:11:33,507] {d:\chicken_fecal_clasification\src\cnnClassifier\utils\common.py:24} INFO - yaml file: params.yaml read successfully
[2024-05-23 12:11:33,509] {d:\chicken_fecal_clasification\src\cnnClassifier\utils\common.py:42} INFO - Created directory: artifacts
[2024-05-23 12:11:33,510] {d:\chicken_fecal_clasification\src\cnnClassifier\utils\common.py:42} INFO - Created directory: artifacts/data_ingestion
[2024-05-23 12:11:40,450] {C:\Users\mfurq\AppData\Local\Temp\ipykernel_21776\3612553148.py:11} INFO - artifacts/data_ingestion/data.zip downlaod! with the folowwing info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
S